# Pure Python Implementation

In [7]:
import math as m 

In [ ]:
# initial set up
input = [1, -1]
w1 = [[1.0, 1.0, 1.0], [-1.0, -1.0, -1.0]]
b1 = 0
w2 = [[1.0, 1.0], [-1.0, -1.0], [-1.0, -1.0]]
b2 = 0
target_class = [1, 0]

In [ ]:
# support functions 

def sigmoid(values: list):
    return [1/(1 + m.exp(-x)) for x in values] 

def deriv_sigmoid(x: list):
    return sigmoid(x)*(1 - sigmoid(x))

def softmax(values: list):
    exp_sum = 0
    for value in values:
        exp_sum += m.exp(value)
    
    return [(m.exp(x)/exp_sum) for x in values]

def deriv_softmax(values: list, ):
    pass


def weighted_sum(input: list, weights:list):
    # initialize empty output list
    output = [0] * len(weights[0])

    # loop over input values
    for idx1, val in enumerate(input):
        # loop over number of weights 
        for idx2 in range(len(weights[0])):
            output[idx2] += val * weights[idx1][idx2]
            
    return output 

def predict(smax_output):
    return target_class[smax_output.index(max(smax_output))]

def forwardpass(input, w1, b1, w2, b2, target):
    # feedforward pass
    k1 = weighted_sum(input, w1, b1) # first linear combination
    h1 = sigmoid(k1) # sigmoid activation function
    k2 = weighted_sum(h1, w2, b2) # second linear combination
    h2 = softmax(k2) # sigmoid activation function
    y = predict(h2)

    return k1, h1, k2, h2, y 

def backprop(input, w1, b1, w2, b2, target):
    # feedforward pass
    k1 = weighted_sum(input, w1, b1) # first linear combination
    h1 = sigmoid(k1) # sigmoid activation function
    k2 = weighted_sum(h1, w2, b2) # second linear combination
    h2 = softmax(k2) # sigmoid activation function

    # backwards pass to compute derivatives
    t_prob = h2[target_class.index(target)] # target probability 
    dL_dy = - 1/t_prob 
    dL_dh2 = dL_dy*(t_prob*(1 - t_prob))
    dL_w2 = 0
    dL_b2 = 0
    dL_h1 = 0
    dL_w1 = 0
    dL_b1 = 0 
    
    return dL_w1, dL_b1, dL_w2, dL_b2

def gradient_desent(w1, b1, w2, b2, dL_w1, dL_b1, dL_w2, dL_b2, alpha):
    
    # update weights in first layer 
    for i in range(len(w1)):
        for idx, w in enumerate(w1[i]):
            w1[i][idx] = w - alpha*dL_w1[i][idx]

    # update weights in second layer 
    for i in range(len(w2)):
         for idx, w in enumerate(w2[i]):
            w2[i][idx] = w - alpha*dL_w2[i][idx]

    return w1, b1, w2, b2 
 

def loss(prediction, target):
    return - m.log(prediction[target_class.index(target)])